In [1]:
# 기본 세팅
import os; import pandas as pd; import numpy as np; import streamlit as st
import matplotlib as plt; import seaborn as sns; import plotly
import re; import requests; from bs4 import BeautifulSoup
from dotenv import load_dotenv; load_dotenv()
from langchain_teddynote import logging   # 랭스미스 로그 추적   
logging.langsmith("19-Streamlit")         # 프로젝트 이름,   set_enable=False : 로그 추적 끄기
from langchain_teddynote.messages import stream_response     # 스트리밍
import warnings; warnings.filterwarnings("ignore")           # 경고 메시지 무시

# LLM models
from langchain_openai import ChatOpenAI                      # 기본 LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")    # 기본 LLM 객체를 정의
from langchain_teddynote.models import MultiModal            # 멀티모달(이미지 인식) 

# LCEL(chain 문법)
from langchain_core.output_parsers import StrOutputParser    # 문자열 출력 파서
from langchain_core.runnables import RunnablePassthrough     # key값 없이 invoke에 값만 넣어줘서 편리.
from langchain_core.runnables import Runnable, RunnableLambda  # 사용자 함수 맵핑
from operator import itemgetter         # invoke에 딕셔너리로 여러 값을 주면 itemgetter가 값들만 추출

# 프롬프트
from langchain_core.prompts import PromptTemplate            # 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate        # AI와 대화용 프롬프트 템플릿
from langchain_core.prompts import load_prompt               # 프롬프트 파일에서 가져오기
from langchain_core.prompts import MessagesPlaceholder       # 임시 공간_대화 기록이 쌓이게 됨
from langchain_core.prompts.few_shot import FewShotPromptTemplate   # 퓨샷 프롬프트 템플릿

# 예제 선택기
from langchain_core.example_selectors import (               # 질문과 유사도가 높은 예제를 선택해서 넣어주기.
    MaxMarginalRelevanceExampleSelector,                     # MMR => 
    SemanticSimilarityExampleSelector,                       # 시맨틱 유사도(cosine similarity) 기반 예제 선택
)

# RAG1. 로드
from langchain_community.document_loaders import PyMuPDFLoader

# RAG2. 청킹
from langchain_text_splitters import RecursiveCharacterTextSplitter

# RAG3. 임베딩
from langchain_openai import OpenAIEmbeddings

# RAG4. 벡터DB
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
import faiss

# 출력 파서
from langchain_core.output_parsers import PydanticOutputParser   # 답변 형식을 원하는대로 정의(ex. JSON)
from pydantic import BaseModel, Field
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser  # tool 파서
from typing import List, Dict, Union, Annotated

# 부가 기능
from langchain_community.utilities import SerpAPIWrapper         # 검색기 SerpAPI
from langchain_teddynote.translate import Translator             # 번역기
from datasets import Dataset                                     # RAG 관련 dataset

# 메모리
from langchain.memory import ConversationBufferMemory            # 대화 내용 저장
from langchain.chains import ConversationChain                   # chain 에서 대화내용 저장
from langchain.docstore import InMemoryDocstore                  # 벡터스토어 요소
from langchain.memory import VectorStoreRetrieverMemory          # VectorStoreRetrieverMemory
from langchain_core.runnables.history import RunnableWithMessageHistory    # 저장된 대화 기록을 가져오는 체인 구성
from langchain_community.chat_message_histories import ChatMessageHistory  # 대화 저장소

# DB
from langchain_community.chat_message_histories import SQLChatMessageHistory   # SQL

# 도큐먼트 로더
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader, PDFPlumberLoader, PyMuPDFLoader   # PDF 로더들
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_community.document_loaders import PyPDFDirectoryLoader      # PDF 디렉토리 로더
from langchain_community.document_loaders.csv_loader import CSVLoader      # CSV 로더
from langchain_community.document_loaders import DirectoryLoader           # 디렉토리 로더
from langchain_community.document_loaders import PythonLoader              # 파이썬 로더

# RAG(LLM) 평가
import langchain
import ragas
from ragas.testset.generator import TestsetGenerator         # 질문 생성
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional  # 질문 형식 : 간단 질문, 추론, 여러 문맥에서 찾는, 조건부 질문
from ragas.llms import LangchainLLMWrapper                                 
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor       # 주요 구문 추출기
from ragas.testset.docstore import InMemoryDocumentStore     # 문서 저장, 관리
from ragas import evaluate
from ragas.metrics import (   # <ragas 평가항목 4가지>
    context_recall,           # 생성된 답변과 검색된 context의 일치도
    context_precision,        # 얼마나 관련성 있는 문서가 상위에 배치되었나
    answer_relevancy,         # 생성된 답변이 질문에 얼마나 유사한지 
    faithfulness,             # 생성된 답변의 사실적 일관성(컨텍스트와 비교)
)

# Tools(Agent)
from langchain_experimental.tools import PythonREPLTool, PythonAstREPLTool  # PythonREPL
from langchain_experimental.utilities import PythonREPL      
from langchain_community.tools.tavily_search import TavilySearchResults  # Tavily 검색 API
from langchain_teddynote.tools import GoogleNews                         # 구글 뉴스기사 검색
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper  # Dall-E API
from IPython.display import Image
from langchain.tools import tool                                         # 사용자 정의 tools
from langchain.agents import create_tool_calling_agent       # LLM Binding Tools
from langchain.agents import AgentExecutor                   # AgentExecutor
from langchain_teddynote.messages import AgentStreamParser   # Agent 중간단계 스트리밍
from langchain.tools.retriever import create_retriever_tool  # RAG 도구화

from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent # Pandas
from langchain.agents.agent_types import AgentType           # Agent Type
from langchain_teddynote.messages import AgentCallbacks      # Agent callback 함수
from langchain_community.agent_toolkits import FileManagementToolkit  # 파일 관리 Toolkit

# Streamlit
import streamlit as st                                       # Streamlit
from langchain_core.messages.chat import ChatMessage         # Streamlit에서 ChatMessage 저장

LangSmith 추적을 시작합니다.
[프로젝트명]
06-DocumentLoader


# RAG chain 구성(기본 구조)

In [ ]:
# RAG 사전준비 4단계(로드-청킹-임베딩-벡터DB)

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()    # docs : pdf 페이지 단위로 쪼개짐 

# # 페이지 내용 출력
# print(docs[10].page_content)
# docs[10].__dict__

# 단계 2: 청킹(Chunking), 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
# print(f"분할된 청크의수: {len(split_documents)}")

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)
# for doc in vectorstore.similarity_search("구글"):
#     print(doc.page_content)

# RAG 실행(Run-time) 4단계

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever(k=4)
# # 검색기에 쿼리를 날려 유사한 chunk 검색 결과를 확인합니다.
# retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean. You must include `page` number in you answer.

# Context: 
{context}

# Question:
{question}

# Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)

삼성전자가 자체 개발한 AI의 이름은 '삼성 가우스'입니다. (page 12)


## Agent 생성

In [ ]:
# 작업 디렉토리 경로 설정
working_directory = "tmp"

# 파일 관리 도구 생성(파일 쓰기, 읽기, 디렉토리 목록 조회)
file_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["write_file", "read_file", "list_directory"],
).get_tools()

In [ ]:
# session_id 를 저장할 딕셔너리 생성
store = {}

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "You are a professional researcher. "
            "You can use the pdf_search tool to search for information in the PDF file. "
            "You can find further information by using search tool. "
            "You can use image generation tool to generate image from text. "
            "Finally, you can use file management tool to save your research result into files.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini")

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:                      # session_id 가 store에 없는 경우
        store[session_ids] = ChatMessageHistory()     # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
    return store[session_ids]                         # 해당 세션 ID에 대한 세션 기록 반환

# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,                       # agent_executor
    get_session_history,                  # 대화 session_id
    input_messages_key="input",           # 프롬프트의 질문이 입력되는 key: "input"
    history_messages_key="chat_history",  # 프롬프트의 메시지가 입력되는 key: "chat_history"
)

agent_stream_parser = AgentStreamParser()